In [258]:
import zipfile
import pandas as pd
import os

In [259]:
###--------------- ETL -------------------#####

data_path = './../data_raw/'
with zipfile.ZipFile(data_path + 'Aires Acondicionados.zip', 'r') as zip_ref:
    zip_ref.extractall('./../data_raw/')

In [260]:
##creates a dictionary of dataframes and store each ac data in a data frame
AC_dataframes = {}
for filename in os.listdir(data_path):
    if filename.startswith('AC') & filename.endswith('.xlsx'):
        file_path = os.path.join(data_path, filename)
        df = pd.read_excel(file_path)
        AC_dataframes[filename] = df
        

for key in AC_dataframes:
    #creates a new column on each dataframe, whose value is the name of the AC unit
    ##crea nueva columna cuyo valor es la unidad de AC
    AC_dataframes[key]['Unidad de AC'] = key[:-5]
    ##erases the first empty rows contained in the original excel files
    #elimina los primeros renglones, y renombra las columnas
    AC_dataframes[key] = AC_dataframes[key].loc[4:].reset_index(drop = True).rename(columns = {'Log Data':'Fecha', 'Unnamed: 1':'Encendido'})
    #changing the format date
    AC_dataframes[key]['Fecha'] = pd.to_datetime(AC_dataframes[key]['Fecha'])
    AC_dataframes[key]['Fecha'] = AC_dataframes[key]['Fecha'].dt.strftime('%b %d, %Y %I:%M:%S %p')

AC_dataframes['AC1.xlsx']

/home/adrian/Documents/ibm-certifications/venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Fecha,Encendido,Unidad de AC
0,"Aug 01, 2025 12:00:00 AM",Off,AC1
1,"Aug 01, 2025 12:10:00 AM",Off,AC1
2,"Aug 01, 2025 12:20:00 AM",Off,AC1
3,"Aug 01, 2025 12:30:01 AM",Off,AC1
4,"Aug 01, 2025 12:40:01 AM",Off,AC1
...,...,...,...
10796,"Oct 14, 2025 11:20:00 PM",Off,AC1
10797,"Oct 14, 2025 11:30:00 PM",Off,AC1
10798,"Oct 14, 2025 11:40:00 PM",Off,AC1
10799,"Oct 14, 2025 11:50:00 PM",Off,AC1


In [261]:
output = pd.DataFrame()
#sorting the keys
sorted_ac = dict(sorted(AC_dataframes.items(), key = lambda x: int(x[0][2:-5]) ) )
# do the union of the dataframes
for key in sorted_ac:
    output = pd.concat([output, sorted_ac[key]], ignore_index=True, axis=0)
output

,Fecha,Encendido,Unidad de AC
0,"Aug 01, 2025 12:00:00 AM",Off,AC1
1,"Aug 01, 2025 12:10:00 AM",Off,AC1
2,"Aug 01, 2025 12:20:00 AM",Off,AC1
3,"Aug 01, 2025 12:30:01 AM",Off,AC1
4,"Aug 01, 2025 12:40:01 AM",Off,AC1
...,...,...,...
129607,"Oct 14, 2025 11:20:00 PM",Off,AC12
129608,"Oct 14, 2025 11:30:00 PM",Off,AC12
129609,"Oct 14, 2025 11:40:00 PM",Off,AC12
129610,"Oct 14, 2025 11:50:00 PM",Off,AC12


In [262]:
##saves the created dataframe into a excel file
#output.to_excel('./../data_output/Base1.xlsx', index=False)

In [263]:
###Base 2

In [264]:
data_path = './../data_raw/'
with zipfile.ZipFile(data_path + 'Energia.zip', 'r') as zip_ref:
    zip_ref.extractall('./../data_raw/')

##reading the files
ener_ac = pd.read_excel(data_path + 'Energia AC.xls')
ener = pd.read_excel(data_path + 'Energía General.xls')

ener_ac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1420 entries, 0 to 1419
Data columns (total 2 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Date & Time                              1420 non-null   object 
 1   AIRE ACONDICIONADO->User Win Total(kWh)  1420 non-null   float64
dtypes: float64(1), object(1)
memory usage: 22.3+ KB


In [265]:
##change names of colums
ener_ac = ener_ac.rename(columns={'Date & Time': 'date_time', 'AIRE ACONDICIONADO->User Win Total(kWh)':'AC_energy(kWh)'})
##changing format of datetime, get rid of seconds
ener_ac['date_time'] = pd.to_datetime(ener_ac['date_time'])
ener_ac['date_time'] = ener_ac['date_time'].dt.strftime('%Y-%m-%d %H:%M')
##reordering from oldest to the more recent
ener_ac = ener_ac.sort_values(by='date_time', ascending=False).reset_index(drop=True)
ener_ac

,date_time,AC_energy(kWh)
0,2025-10-15 08:00,45.38
1,2025-10-15 06:00,105.46
2,2025-10-15 05:00,37.74
3,2025-10-15 04:00,43.32
4,2025-10-15 03:00,43.76
...,...,...
1415,2025-08-17 08:00,71.73
1416,2025-08-17 07:00,145.55
1417,2025-08-17 06:00,108.56
1418,2025-08-17 05:00,23.12


In [266]:
#same steps as above for the general energy data
ener = ener.rename(columns={'Date & Time': 'date_time', 'Medición General->User Win Total(kWh)': 'general_energy(kWh)'})
ener['date_time'] = pd.to_datetime(ener['date_time'])
ener['date_time'] = ener['date_time'].dt.strftime('%Y-%m-%d %H:%M')
ener = ener.sort_values(by='date_time', ascending=False).reset_index(drop=True)
ener

,date_time,general_energy(kWh)
0,2025-10-15 09:00,163.06
1,2025-10-15 08:00,95.28
2,2025-10-15 06:00,103.42
3,2025-10-15 05:00,36.85
4,2025-10-15 04:00,43.63
...,...,...
6568,2025-01-12 23:00,126.39
6569,2025-01-12 22:00,124.89
6570,2025-01-12 21:00,129.52
6571,2025-01-12 20:00,121.67


In [267]:
##joins the ac_energy and general_energy dataframes for values on the same date_time.
##since the general energy (ener) has more observations, left join is done on it
output2 = ener.merge(ener_ac, on='date_time', how='left')
output2

,date_time,general_energy(kWh),AC_energy(kWh)
0,2025-10-15 09:00,163.06,NaN
1,2025-10-15 08:00,95.28,45.38
2,2025-10-15 06:00,103.42,105.46
3,2025-10-15 05:00,36.85,37.74
4,2025-10-15 04:00,43.63,43.32
...,...,...,...
6568,2025-01-12 23:00,126.39,NaN
6569,2025-01-12 22:00,124.89,NaN
6570,2025-01-12 21:00,129.52,NaN
6571,2025-01-12 20:00,121.67,NaN


In [268]:
##saves created dataframe into a excel file
#output2.to_excel('./../data_output/Base2.xlsx', index=False)

In [270]:
##temp ext data
temp_ext = pd.read_excel(data_path + 'Temp Ext.xlsx')
#rename columns
temp_ext = temp_ext.rename(columns = {'Log Data':'date_time', 'Unnamed: 1': 'celcius'})
##get rid of first empty rows
temp_ext = temp_ext[4:].reset_index(drop=True)
#change format
temp_ext['date_time'] = pd.to_datetime(temp_ext['date_time'])
temp_ext['date_time'] = temp_ext['date_time'].dt.strftime('%Y-%m-%d %H:%M')
temp_ext
#export to excel file
#temp_ext.to_excel('./../data_output/temperature.xlsx', index=False)

,date_time,celcius
0,2025-08-01 00:00,25.67
1,2025-08-01 00:10,25.56
2,2025-08-01 00:20,25.81
3,2025-08-01 00:30,25.56
4,2025-08-01 00:40,25.48
...,...,...
10796,2025-10-14 23:20,21.95
10797,2025-10-14 23:30,22.15
10798,2025-10-14 23:40,22.2
10799,2025-10-14 23:50,22.3


In [271]:
##merge temperatur with energy table (base2) using times that appear both in base2 and temp_ext
output3 = output2.merge(temp_ext, on='date_time', how='inner')
#output3.to_excel('./../data_output/Base3.xlsx', index=False)
output3

,date_time,general_energy(kWh),AC_energy(kWh),celcius
0,2025-10-15 00:00,40.58,41.41,22.3
1,2025-10-14 23:00,91.01,29.26,21.45
2,2025-10-14 22:00,220.42,0.21,21.62
3,2025-10-14 21:00,277.80,0.26,22.69
4,2025-10-14 20:00,355.56,167.92,24.61
...,...,...,...,...
1796,2025-08-01 04:00,151.13,NaN,22.3
1797,2025-08-01 03:00,161.99,NaN,22.38
1798,2025-08-01 02:00,161.54,NaN,24.96
1799,2025-08-01 01:00,170.10,NaN,25.53
